In [1]:
import pickle
import pathlib

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)
clean_data_path = DATA_DIR / 'processed' / 'ames_clean_eng.pkl'
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

c:\Users\Pedro Antônio Silva\Desktop\Semestre 4\Machine Learning\proj_1\ames\data


In [27]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

model_data = data.copy()

y = model_data["SalePrice"].copy()
X = model_data.drop(columns=["SalePrice"]).copy()
        
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=12,)

piped_model = Pipeline([
    ("Scaler", StandardScaler()),
    ("Ridge", Ridge())
])

param_grid_r = {
    'Ridge__alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
}

grid_search_r = GridSearchCV(piped_model, param_grid_r, 
                              cv=5, scoring="neg_mean_squared_error")

grid_search_r.fit(Xtrain,ytrain)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Scaler', StandardScaler()),
                                       ('Ridge', Ridge())]),
             param_grid={'Ridge__alpha': [0.001, 0.01, 0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error')

In [28]:
grid_search_r.best_params_

{'Ridge__alpha': 10.0}

In [31]:
score = cross_val_score(grid_search_r.best_estimator_,Xtrain,ytrain, scoring="neg_mean_squared_error")
score = np.sqrt(-score)
score

array([0.04619468, 0.05505818, 0.05658743, 0.04991424, 0.05234268])

In [32]:
error_percent = 100 * (10**score - 1)
error_percent

array([11.2230191 , 13.51628878, 13.91670932, 12.1796907 , 12.80872329])

In [33]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

model_data = data.copy()

y = model_data["SalePrice"].copy()
X = model_data.drop(columns=["SalePrice"]).copy()
        
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=12,)

piped_model2 = Pipeline([
    ("Scaler", StandardScaler()),
    ("Lasso", Lasso())
])

param_grid_l = {
    'Lasso__alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
}

grid_search_l = GridSearchCV(piped_model2, param_grid_l, 
                              cv=5, scoring="neg_mean_squared_error")

grid_search_l.fit(Xtrain, ytrain)

score2 = cross_val_score(grid_search_l.best_estimator_, Xtrain, ytrain, scoring="neg_mean_squared_error")
score2 = np.sqrt(-score2)

In [34]:
grid_search_l.best_params_

{'Lasso__alpha': 0.001}

In [35]:
error_percent2 = 100 * (10**score2 - 1)
error_percent2

array([10.82699101, 13.6761723 , 14.31409906, 11.3697923 , 13.23045955])

In [36]:
from scipy.stats import ttest_ind

ttest_ind(error_percent,error_percent2, equal_var=False)

Ttest_indResult(statistic=0.05478940639817598, pvalue=0.957791573662176)

como pvalue tem um valor alto (maior que 5%, convenc) não podemos rejeitar a hipótese nula, portanto não conseguimos afirmar que um modelo e superior ao outro. 